In [1]:
import numpy as np
import pandas as pd

In [2]:
turb_data=pd.read_csv('turb_data.csv')
turb_data

,x_coord,y_coord,ref_wind_speed,wind_speed,turb_num,num_tot_turb,ID,IDnum
0,0.000000,0.000000,2.980920,4,0,2,2_turbine_248.0_degrees_558_meters,0
1,517.368591,209.030479,2.990576,4,1,2,2_turbine_248.0_degrees_558_meters,0
2,0.000000,0.000000,2.713630,4,0,2,2_turbine_269.0_degrees_279_meters,1
3,278.957507,4.869221,2.983835,4,1,2,2_turbine_269.0_degrees_279_meters,1
4,0.000000,0.000000,2.989921,4,0,2,2_turbine_289.0_degrees_558_meters,2
...,...,...,...,...,...,...,...,...
8519,1304.330478,148.000000,12.702296,14,1,3,3_turbine_65th_simulation,3770
8520,453.623822,-12.000000,11.295371,14,2,3,3_turbine_65th_simulation,3770
8521,0.000000,0.000000,12.662241,14,0,3,3_turbine_71th_simulation,3771
8522,2704.096313,-157.000000,11.475002,14,1,3,3_turbine_71th_simulation,3771


In [3]:
dataset=turb_data.drop(['ID','turb_num'], axis=1)

In [4]:
dataset

,x_coord,y_coord,ref_wind_speed,wind_speed,num_tot_turb,IDnum
0,0.000000,0.000000,2.980920,4,2,0
1,517.368591,209.030479,2.990576,4,2,0
2,0.000000,0.000000,2.713630,4,2,1
3,278.957507,4.869221,2.983835,4,2,1
4,0.000000,0.000000,2.989921,4,2,2
...,...,...,...,...,...,...
8519,1304.330478,148.000000,12.702296,14,3,3770
8520,453.623822,-12.000000,11.295371,14,3,3770
8521,0.000000,0.000000,12.662241,14,3,3771
8522,2704.096313,-157.000000,11.475002,14,3,3771


In [5]:
import three_desc_model as desc
from cutoffs import Polynomial
import matplotlib.pyplot as plt
import GPy
from IPython.display import display
symbol="Siemens"
turb = [symbol]
numsims=dataset["IDnum"].iloc[-1]+1

In [6]:
nl=desc.NeighborlistCalculator(cutoff=4001,cone_grad=0.12582561117875557, cone_offset=72.24947126849844)
Gs = {"Siemens": [{"type":"G2", "turbine":"Siemens","eta":4.25387599, "offset": 1.0151402},
                  {"type":"G4", "elements":"Siemens","eta":2.56450515, "gamma":8.04475192, "zeta": 2.5356155},
                  {"type":"G6", "elements":"Siemens","eta":2.33043463, "gamma": 0.50753377, "zeta": 0.93372721}
                 ]}
finpr=desc.FingerprintCalculator(cutoff=4001,Gs=Gs,Rct=3000,delta_R=100,cone_grad=0.12582561117875557, cone_offset=72.24947126849844)
count=0
dataset_fp=np.empty(shape=(0, 3))
dataset_rws=np.empty(shape=(0, 1))
dataset_ws=np.empty(shape=(0, 1))
dataset_pos=np.empty(shape=(0, 2))
for i in range(numsims):
    numturb=dataset["num_tot_turb"].iloc[count]
    position = np.empty((numturb,2))
    rws = np.empty((numturb,1))
    ws = np.empty((numturb,1))
    fp = np.empty((numturb,3))
    for k in range(numturb):
        position[k,0]=dataset["x_coord"].iloc[count]#dataset_full.at[count,"x_coord"]
        position[k,1]=dataset["y_coord"].iloc[count]#dataset_full.at[count,"y_coord"]
        rws[k,0]=dataset["ref_wind_speed"].iloc[count]#dataset_full.at[count,"ref_wind_speed"]
        ws[k,0]=dataset["wind_speed"].iloc[count]#dataset_full.at[count,"wind_speed"]
        count = count+ 1
    neigh=nl.calculate(turb*numturb,position)
    fingerprints=finpr.calculate(turb*numturb,position,neigh,symbol)
    fingerprints=np.array(fingerprints)
    dataset_fp=np.append(dataset_fp,fingerprints,axis=0)
    dataset_rws=np.append(dataset_rws,rws,axis=0)
    dataset_ws=np.append(dataset_ws,ws,axis=0)
    dataset_pos=np.append(dataset_pos,position,axis=0)
all_data=np.concatenate((dataset_fp, dataset_ws, dataset_rws),axis=1)
all_data = pd.DataFrame(all_data, columns = ['Fingerprint(G2)','Fingerprint(G4)','Fingerprint(G6)','Wind_Speed','Ref_Wind_Speed'])
X=all_data[["Fingerprint(G2)","Fingerprint(G4)","Fingerprint(G6)","Wind_Speed"]].to_numpy()
Y=all_data[["Ref_Wind_Speed"]].to_numpy()
train_dataset = all_data.sample(frac=0.8, random_state=0)
test_dataset = all_data.drop(train_dataset.index)
Xtrain=train_dataset[["Fingerprint(G2)","Fingerprint(G4)","Fingerprint(G6)","Wind_Speed"]].to_numpy()
Ytrain=train_dataset[["Ref_Wind_Speed"]].to_numpy()
Xtest=test_dataset[["Fingerprint(G2)","Fingerprint(G4)","Fingerprint(G6)","Wind_Speed"]].to_numpy()
Ytest=test_dataset[["Ref_Wind_Speed"]].to_numpy()

In [7]:
# define kernel
ker = GPy.kern.RBF(4,lengthscale=0.1, ARD=True)# + GPy.kern.White(2)

In [8]:
# create simple GP model
m = GPy.models.GPRegression(Xtrain,Ytrain,ker)

In [9]:
# optimize and plot
m.optimize(messages=True,max_f_eval = 1000)
#fig = m.plot()
#display(GPy.plotting.show(fig, filename='basic_gp_regression_notebook_2d'))
display(m)

TypeError: unsupported format string passed to NoneType.__format__

In [ ]:
print(m)

In [ ]:
slices = [0.2, 0.5, 0.8, 1.0]
figure = GPy.plotting.plotting_library().figure(4, 1)
                            
for i, y in zip(range(4), slices):
    canvas = m.plot(figure=figure, fixed_inputs=[(1,y)], row=(i+1), plot_data=False)
#GPy.plotting.show(canvas, filename='basic_gp_regression_notebook_slicing')

In [ ]:
[Ypred,Yvar]=m.predict(Xtest)
error_test=np.abs(Ypred-Ytest)
np.max(error_test)

In [ ]:
[Yval,Yvalvar]=m.predict(Xtrain)
error_train=np.abs(Yval-Ytrain)
np.max(error_train)

In [ ]:
[Yall,Yallvar]=m.predict(X)
error_all=np.abs(Yall-Y)
np.max(error_all)

In [ ]:
position=np.array([[0,0],[500,0]])
neigh=nl.calculate(turb*2,position)
#print(neigh)
fingerprints=np.array(finpr.calculate(turb*2,position,neigh,symbol))
test_data=np.append(fingerprints,[[10],[10]],axis=1)
#print(test_data)
m.predict(test_data)